In [3]:
# https://molecularneurodegeneration.biomedcentral.com/articles/10.1186/s13024-022-00589-x
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE175546

In [1]:
import scanpy as sc
import os
import glob

In [2]:
data_dir = "/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau"
h5_files = sorted(glob.glob(os.path.join(data_dir, "*.h5")))

In [4]:
h5_files

['/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341444_TL1_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341445_TL2_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341446_TL3_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341447_TL4_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341448_TL9_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341449_TL10_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341450_TL11_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341451_TL12_raw_feature_bc_matrix.h5',
 '/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/GSM5341452_TL13_raw_feature_bc_matrix.h5

In [5]:
tl_mapping = {
    'TL1':  {'age': '12 months', 'area': 'Cortex', 'condition': 'WT',          'sex': 'Female'},
    'TL2':  {'age': '12 months', 'area': 'Cortex', 'condition': 'Tau4RΔK',     'sex': 'Female'},
    'TL3':  {'age': '12 months', 'area': 'Cortex', 'condition': 'APP;PS1',     'sex': 'Female'},
    'TL4':  {'age': '12 months', 'area': 'Cortex', 'condition': 'Tau4RΔK-AP',  'sex': 'Female'},
    'TL9':  {'age': '12 months', 'area': 'Cortex', 'condition': 'WT',          'sex': 'Female'},
    'TL10': {'age': '12 months', 'area': 'Cortex', 'condition': 'Tau4RΔK',     'sex': 'Female'},
    'TL11': {'age': '12 months', 'area': 'Cortex', 'condition': 'APP;PS1',     'sex': 'Female'},
    'TL12': {'age': '12 months', 'area': 'Cortex', 'condition': 'Tau4RΔK-AP',  'sex': 'Female'},
    'TL13': {'age': '6 months',  'area': 'Cortex', 'condition': 'WT',          'sex': 'Female'},
    'TL14': {'age': '6 months',  'area': 'Cortex', 'condition': 'Tau4RΔK',     'sex': 'Female'},
    'TL15': {'age': '6 months',  'area': 'Cortex', 'condition': 'APP;PS1',     'sex': 'Female'},
    'TL16': {'age': '6 months',  'area': 'Cortex', 'condition': 'Tau4RΔK-AP',  'sex': 'Female'},
}

In [13]:
os.path.basename(file).split('_')[1].replace('.h5', '')

'TL16'

In [14]:
adatas = []

for file in h5_files:
    tl_id = os.path.basename(file).split('_')[1].replace('.h5', '')

    # Load data
    adata = sc.read_10x_h5(file)
    adata.var_names_make_unique()
    adata.obs['sample'] = tl_id

    # Add metadata
    meta = tl_mapping.get(tl_id, {})
    for key in ['age', 'area', 'condition', 'sex']:
        adata.obs[key] = meta.get(key, 'unknown')

    adatas.append(adata)

/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/zihe.zheng/miniconda3/envs/scanpy_dask/lib/python3.13/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/icb/zihe.zheng/miniconda3/

In [16]:
combined_adata = adatas[0].concatenate(
    adatas[1:], 
    batch_key='batch', 
    batch_categories=[adata.obs['sample'][0] for adata in adatas]
)

combined_adata.write("/lustre/groups/ml01/projects/2024_microglia_zihe.zheng/mouse_abtau/combined_data.h5ad")

/tmp/ipykernel_1133905/2940053776.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  batch_categories=[adata.obs['sample'][0] for adata in adatas]
/tmp/ipykernel_1133905/2940053776.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  combined_adata = adatas[0].concatenate(


In [18]:
combined_adata.obs

,sample,age,area,condition,sex,batch
AAACCCAAGAAACACT-1-TL1,TL1,12 months,Cortex,WT,Female,TL1
AAACCCAAGAAACCAT-1-TL1,TL1,12 months,Cortex,WT,Female,TL1
AAACCCAAGAAACCCA-1-TL1,TL1,12 months,Cortex,WT,Female,TL1
AAACCCAAGAAACCCG-1-TL1,TL1,12 months,Cortex,WT,Female,TL1
AAACCCAAGAAACCTG-1-TL1,TL1,12 months,Cortex,WT,Female,TL1
...,...,...,...,...,...,...
TTTGTTGTCTTTGCTA-1-TL16,TL16,6 months,Cortex,Tau4RΔK-AP,Female,TL16
TTTGTTGTCTTTGCTG-1-TL16,TL16,6 months,Cortex,Tau4RΔK-AP,Female,TL16
TTTGTTGTCTTTGGAG-1-TL16,TL16,6 months,Cortex,Tau4RΔK-AP,Female,TL16
TTTGTTGTCTTTGGCT-1-TL16,TL16,6 months,Cortex,Tau4RΔK-AP,Female,TL16


In [21]:
combined_adata.obs.condition.value_counts()

condition
APP;PS1       20384640
Tau4RΔK       20384640
Tau4RΔK-AP    20384640
WT            20384640
Name: count, dtype: int64

In [24]:
combined_adata.obs['sample'].value_counts()

sample
TL1     6794880
TL2     6794880
TL3     6794880
TL4     6794880
TL9     6794880
TL10    6794880
TL11    6794880
TL12    6794880
TL13    6794880
TL14    6794880
TL15    6794880
TL16    6794880
Name: count, dtype: int64

In [19]:
combined_adata.X.data

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [20]:
combined_adata

AnnData object with n_obs × n_vars = 81538560 × 31053
    obs: 'sample', 'age', 'area', 'condition', 'sex', 'batch'
    var: 'gene_ids', 'feature_types', 'genome'